# Journey to LLM and prompt engineering

## The Chat Format

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

### Setup

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')


In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]


#### "temperature=1" means maximum diverse and creative!  Each time the response would be totally different.

In [4]:
messages =  [  
{'role':'system', 'content':'你是一個說話像莎士比亞的助手。'},    
{'role':'user', 'content':'告訴我一個笑話'},   
{'role':'assistant', 'content':'為什麼雞要過馬路'},   
{'role':'user', 'content':'我不知道'}  ]

response = get_completion_from_messages(messages, temperature=1)
print(response)


好吧，讓我為您帶來一則歡樂的笑話：

在古老的費爾登王國裡，有一頭非常聰明的獅子。有一天，獅子走進一家酒館，看到一隻雞正在有趣地舞動著。獅子走過去問道：「雞啊，你為什麼要過馬路呢？」

雞停下舞蹈，深思片刻，然後回答說：「親愛的獅子，我過馬路是因為我的劇本要求我在酒館對面的戲劇場演出。這檔戲是一個莎士比亞風格的喜劇，我飾演一個幽默的雞角色。所以，我必須過馬路才能按時上台演出。」

獅子聽後大為驚訝，笑著說道：「噢！那是我無法預料的答案！祝你在舞台上演出得非常成功，我的朋友！」

雞感激地向獅子點頭示意，然後繼續舞動，唱著莎士比亞風格的歌謠，讓整個酒館充滿了歡樂的氛圍。

希望這則笑話能為您帶來歡樂！


In [5]:
messages =  [  
{'role':'system', 'content':'你是友善的聊天機器人。'},    
{'role':'user', 'content':'嗨，我的名字是Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)


嗨Isa！很高興認識你。有什麼我可以幫助你的呢？


### Didn't mention names.

In [6]:
messages =  [  
{'role':'system', 'content':'你是友善的聊天機器人。'},    
{'role':'user', 'content':'是的，你能提醒我，我的名字是什麼嗎？'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)


當然可以！但我無法知道您的本名，因為我只是一個聊天機器人。但您可以告訴我您的名字，我將盡力記住並在以後提醒您。


In [7]:
messages =  [  
{'role':'system', 'content':'你是友善的聊天機器人。'},
{'role':'user', 'content':'嗨，我的名字是Isa'},
{'role':'assistant', 'content': "嗨，Isa！很高興見到你。今天有什麼我可以幫助你的嗎？"},
{'role':'user', 'content':'是的，你可以提醒我，我的名字是什麼？'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)


當然！你的名字是Isa。我會記住這個名字，如果你有任何需要，隨時告訴我。


## Chain-of-thought prompting

In [8]:
delimiter = "####"
system_message = f"""
按照以下步驟來回答客戶的查詢。
客戶的查詢將以四個井號來界定，也就是 {delimiter}。

步驟一：{delimiter} 首先判斷使用者是否在詢問有關特定產品或多種產品的問題。產品類別不計入。

步驟二：{delimiter} 如果使用者正在詢問有關特定產品的問題，請識別這些產品是否在以下清單中。

所有可用產品：
1.
產品：TechPro Ultrabook
類別：電腦與筆記型電腦
品牌：TechPro
型號：TP-UB100
保固：1 年
評價：4.5
功能：13.3 英寸顯示器，8GB RAM，256GB SSD，Intel Core i5處理器
描述：適合日常使用的纖薄輕巧的超級筆記本電腦。
價格：$799.99

2.
產品：BlueWave Gaming Laptop
類別：電腦與筆記型電腦
品牌：BlueWave
型號：BW-GL200
保固：2 年
評價：4.7
功能：15.6 英寸顯示器，16GB RAM，512GB SSD，NVIDIA GeForce RTX 3060
描述：提供身臨其境體驗的高性能遊戲筆記本電腦。
價格：$1199.99"

3. 
產品：PowerLite Convertible
類別：電腦與筆記型電腦
品牌：PowerLite
型號：PL-CV300
保固：1 年
評價：4.3
功能：14 英寸觸控螢幕，8GB RAM，256GB SSD，360 度轉軸
描述：靈活多變的可轉換筆記本電腦，具有反應靈敏的觸控螢幕。
價格：$699.99

4.
產品：TechPro Desktop
類別：電腦與筆記型電腦
品牌：TechPro
型號：TP-DT500
保固：1 年
評價：4.4
功能：Intel Core i7 處理器，16GB RAM，1TB HDD，NVIDIA GeForce GTX 1660
描述：適用於工作和娛樂的強大桌面電腦。
價格：$999.99"

5. 
產品：BlueWave Chromebook
類別：電腦與筆記型電腦
品牌：BlueWave
型號：BW-CB100
保固：1 年
評價：4.1
功能：11.6 英寸顯示器，4GB RAM，32GB eMMC，Chrome OS
描述：適合日常任務的緊湊型且價格適中的 Chromebook。
價格：$249.99

步驟三：{delimiter} 如果訊息中包含上述清單中的產品，列出使用者在他們的訊息中可能做出的任何假設，\
例如，認為筆記本電腦 X 比筆記本電腦 Y 大，或者認為筆記本電腦 Z 有2年保固。

步驟四：{delimiter} 如果使用者做出了任何假設，根據您的產品資訊判定這個假設是否正確。

步驟五：{delimiter} 首先，如果適用，禮貌地糾正客戶的不正確假設。\
只提及或參考上述5個可用產品清單中的產品，因為這些是商店唯一出售的5種產品。以友善的語調回答客戶。

請使用以下格式：
步驟一：{delimiter} <步驟一的推理>
步驟二：{delimiter} <步驟二的推理>
步驟三：{delimiter} <步驟三的推理>
步驟四：{delimiter} <步驟四的推理>
對使用者的回應：{delimiter} <對客戶的回應>

確保在每個步驟之間包含 {delimiter} 進行分隔。
"""


In [9]:
user_message = f"""
BlueWave Chromebook 比 TechPro Desktop 貴多少錢。請用中文回答。"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)


步驟一：使用者正在詢問有關特定產品的問題。

步驟二：使用者提到了兩個產品，分別是 BlueWave Chromebook 和 TechPro Desktop。

步驟三：使用者假設 BlueWave Chromebook 比 TechPro Desktop 貴。

步驟四：根據產品資訊，TechPro Desktop 的價格是 $999.99，而 BlueWave Chromebook 的價格是 $249.99。因此，使用者的假設是正確的，BlueWave Chromebook 比 TechPro Desktop 更便宜。

對使用者的回應：根據產品資訊，BlueWave Chromebook 比 TechPro Desktop 更便宜，價格差異為 $750。如果您對這兩款產品有任何進一步的問題，請隨時告訴我們。


## OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [11]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [12]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
你是OrderBot，一種自動收集披薩餐廳訂單的服務。\
你首先向顧客問好，然後收集訂單，然後問是否自取或送貨。\
你等著收集整個訂單，然後總結它，並最後確認一次顧客是否想添加其他東西。\
如果是送貨，你會詢問地址。最後你收取現金付款。\
請確定澄清所有選項、附加品和尺寸以便從菜單中唯一地識別出項目。\
你以簡短的、非常口語化友好的風格回答。\
菜單包括
辣肉香腸披薩 12.95, 10.00, 7.00
起司披薩   10.95, 9.25, 6.50
茄子披薩   11.95, 9.75, 6.75
薯條 4.50, 3.50
希臘沙拉 7.25
配料:
額外的起司 2.00,
蘑菇 1.50
香腸 3.00
加拿大煙肉 3.50
AI醬 1.50
辣椒 1.00
飲料:
可口可樂 3.00, 2.00, 1.00
雪碧 3.00, 2.00, 1.00
瓶裝水 5.00
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="你好", placeholder='在這裡輸入文字…')
button_conversation = pn.widgets.Button(name="聊天!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard


/var/folders/7t/drc_5mxn3c96k2_3zc1n57yh0000gn/T/ipykernel_60671/3082588398.py:10: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.1, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))


Column
    [0] TextInput(placeholder='在這裡輸入文字…')
    [1] Row
        [0] Button(name='聊天!')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

In [13]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'建立前一個食品訂單的json摘要。列出每項物品的價格。 \
應有的欄位為1)比薩，包含尺寸 2)配料列表 3)飲料列表，包含尺寸 4)配菜列表，包含尺寸 5)總價 '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)


以下是前一個食品訂單的JSON摘要：

{
  "比薩": {
    "尺寸": "中",
    "價格": 10.95
  },
  "配料列表": [
    "辣椒"
  ],
  "飲料列表": [
    {
      "名稱": "可口可樂",
      "尺寸": "大",
      "價格": 3.00
    }
  ],
  "配菜列表": [],
  "總價": 13.95
}


## Mandarin Idiom Explainer

The notebook contains snippets from both CS50 and Andrew Ng Prompting tutorials:

https://www.youtube.com/watch?v=vw-KWfKwvTQ&t=286s
https://github.com/yu-to-chen/generative-ai/blob/main/langchain/7-chatbot_Chinese.ipynb
It is a primitive chatbot capable of responding to every situation with a Chinese 成語 and provide a word of encouragement for learning languages.

In [14]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
你是中文成語機器人，一位親切且樂於助人的中文老師。
你首先透過解釋適合每種情況的成語來回應。
然後提供關於學習語言的鼓勵性建議。
"""} ]  # accumulate messages


In [15]:
inp = pn.widgets.TextInput(value="你好", placeholder='在這裡輸入文字…')
button_conversation = pn.widgets.Button(name="聊天!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard


/var/folders/7t/drc_5mxn3c96k2_3zc1n57yh0000gn/T/ipykernel_60671/3082588398.py:10: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.1, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))


Column
    [0] TextInput(placeholder='在這裡輸入文字…')
    [1] Row
        [0] Button(name='聊天!')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)